# Extract

In [ ]:
# Importing libraries
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
from google.colab import drive


In [22]:
drive.mount('/content/drive')
df_artist=pd.read_csv('/content/drive/MyDrive/artists_mod.csv')
df_tracks = pd.read_csv('/content/drive/MyDrive/tracks_mod.csv')
print(df_artist.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(1162095, 5)


# Transform

## Tracks

In [23]:
df_tracks = df_tracks[df_tracks['popularity'].notna()] ## KEEP TRACKS WITH POPULARITY
df_tracks.dropna(inplace=True) # quitar los NaN
df_tracks['duration_ms']=df_tracks['duration_ms']/1000 # Tener duración en segundos

In [24]:
df_tracks.rename(columns={"duration_ms": "duration_s"}, inplace=True) # cambiar nombre columna

In [25]:
df_tracks.dropna(inplace=True) # quita las canciones con vacios para asegurar calidad de datos
df_tracks.drop_duplicates(inplace=True) # borrar canciones duplicadas

In [26]:
variables=['id','name','popularity','duration_s','explicit','id_artists','danceability','energy','loudness','speechiness','acousticness','tempo'] # se eligen las variables deseadas
df_tracks=df_tracks[variables] # selección de variables a utilizar

In [27]:
from sklearn import preprocessing
import ast

df_tracks['loudness']=abs(df_tracks['loudness']) # se obtiene el valor absoluto del loudness
df_tracks_ids= df_tracks[['id','name','id_artists']] # se obtiene un nuevo dataframe con los datos deseados

#min_max_scaler = preprocessing.MinMaxScaler() # se normaliza para subir a postgresql
#df_tracks_norm = min_max_scaler.fit_transform(df_tracks[['popularity','duration_s','explicit','danceability','energy','loudness','speechiness','acousticness','tempo']])
df_tracks_norm = df_tracks[['popularity','duration_s','explicit','danceability','energy','loudness','speechiness','acousticness','tempo']]
df_tracks_norm = pd.DataFrame(df_tracks_norm, columns=['popularity','duration_s','explicit','danceability','energy','loudness','speechiness','acousticness','tempo'])
df_tracks_final= pd.merge(df_tracks_ids, df_tracks_norm, left_index=True, right_index=True) ## este es el que se exporta


df_tracks_final['id_artists']=df_tracks_final["id_artists"].apply(ast.literal_eval)
df_tracks_final=df_tracks_final.explode('id_artists') # una canción por artista


df_tracks_final.rename(columns={"name": "name_tracks"}, inplace = True)
df_tracks_final['id_artists'] = df_tracks_final['id_artists'].replace({'\'': ''},regex=True).replace({"'": ''},regex=True)
df_tracks_final['name_tracks'] = df_tracks_final['name_tracks'].replace({'\'': ''},regex=True).replace({"'": ''},regex=True)

df_tracks_final

<ipython-input-27-2da3dfe22e80>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tracks['loudness']=abs(df_tracks['loudness']) # se obtiene el valor absoluto del loudness


,id,name_tracks,id_artists,popularity,duration_s,explicit,danceability,energy,loudness,speechiness,acousticness,tempo
0,35iwgR4jXetI318WEWsa1Q,Carve,45tIt06XoI0Iio4LBEVpls,6.0,126.903,0,0.645,0.445,13.338,0.4510,0.674,104.851
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,14jtPCOoNZwquk5wd9DxrY,0.0,98.200,0,0.695,0.263,22.136,0.9570,0.797,102.009
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,5LiOoJbxVSAMkBS2fUm3X2,0.0,181.640,0,0.434,0.177,21.180,0.0512,0.994,130.418
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,3BiJGZsyX9sJchTqcSA7Su,0.0,163.080,0,0.402,0.158,16.900,0.0390,0.989,103.220
5,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,3BiJGZsyX9sJchTqcSA7Su,0.0,178.933,0,0.227,0.261,12.343,0.0382,0.994,118.891
...,...,...,...,...,...,...,...,...,...,...,...,...
586664,1Kzjk1EyngBcP4T8x3fyqv,同行 (新加坡電視劇《愛...沒有距離》主題曲),6PWJWwEm8BSBFAIAUWlwe4,43.0,205.238,0,0.743,0.679,3.952,0.0323,0.269,126.070
586667,5rgu12WBIHQtvej2MdHSH0,云与海,1QLBXKM5GCpyQQSVMNZqrZ,50.0,258.267,0,0.560,0.518,7.471,0.0292,0.785,131.896
586668,0NuWgxEp51CutD2pJoF4OM,blind,1dy5WNgIKQU6ezkpZs4y8z,72.0,153.293,0,0.765,0.663,5.223,0.0652,0.141,150.091
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,4jGPdu95icCKVF31CcFKbS,58.0,142.003,0,0.696,0.615,6.212,0.0345,0.206,90.029


## Géneros

In [28]:
import ast

# Aplicar ast.literal_eval() a cada fila de la columna "mi_columna"
df_artist["genres"] = df_artist["genres"].apply(ast.literal_eval)
df3 = pd.DataFrame(df_artist['genres'].to_list())
df3 = df3[0]
df_genres = pd.concat([df_artist, df3], axis=1)
df_genres= df_genres[[0]]
df_genres= df_genres.drop_duplicates(subset=[0], keep = 'first')
df_genres.rename(columns={0: "genero"}, inplace = True)
df_genres.dropna(inplace = True)
df_genres['genre_id'] =range(1, len(df_genres)+1) #df_genres[['genero']].sum(axis=1).map(hash).abs()
df_genres=df_genres[['genre_id','genero']]
df_genres['genero'] = df_genres['genero'].replace({'\'': ''},regex=True)

<ipython-input-28-1d38fc252c1f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genres.rename(columns={0: "genero"}, inplace = True)
<ipython-input-28-1d38fc252c1f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genres.dropna(inplace = True)
<ipython-input-28-1d38fc252c1f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genres['genre_id'] =range(1, len(df_genres)+

In [29]:
#df_genres['genre_id'].astype(int)
df_genres

,genre_id,genero
45,1,carnaval cadiz
136,2,classical harp
141,3,classical contralto
142,4,british choir
144,5,childrens story
...,...,...
816992,5359,thrash metal
840273,5360,ugandan traditional
1045510,5361,viking metal
1118346,5362,metal piauiense


In [30]:
# Aplicar ast.literal_eval() a cada fila de la columna "mi_columna"
df_artists = pd.concat([df_artist, df3], axis=1)
df_artists.rename(columns={0: "genero"}, inplace = True)
df_artists = df_artists[df_artists['genero'].notna()]
df_artists.drop(['genres'], axis = 1, inplace= True) # remove column (genres)
df_artists

,id,followers,name,popularity,genero
45,0VLMVnVbJyJ4oyZs2L3Yl2,71.0,Las Viudas De Los Bisabuelos,6,carnaval cadiz
46,0dt23bs4w8zx154C5xdVyl,63.0,Los De Capuchinos,5,carnaval cadiz
47,0pGhoB99qpEJEsBQxgaskQ,64.0,Los “Pofesionales”,7,carnaval cadiz
48,3HDrX2OtSuXLW5dLR85uN3,53.0,Los Que No Paran De Rajar,6,carnaval cadiz
136,22mLrN5fkppmuUPsHx6i2G,59.0,Vera Dulova,3,classical harp
...,...,...,...,...,...
1162087,0vZbvoyHDSwTGY4v1oUo31,1682.0,Josh Sneed,20,clean comedy
1162088,3V4ODgUObsEHFX6GQUy8xl,419.0,Matt McCarthy,9,deep comedy
1162090,3cOzi726Iav1toV2LRVEjp,4831.0,Ali Siddiq,34,black comedy
1162093,5nvjpU3Y7L6Hpe54QuvDjy,2357.0,Donnell Rawlings,15,black comedy


In [31]:
df_artists=pd.merge(df_artists,df_genres, on='genero', how='left')
df_artists = df_artists[df_artists['genre_id'].notna()]
df_artists

,id,followers,name,popularity,genero,genre_id
0,0VLMVnVbJyJ4oyZs2L3Yl2,71.0,Las Viudas De Los Bisabuelos,6,carnaval cadiz,1.0
1,0dt23bs4w8zx154C5xdVyl,63.0,Los De Capuchinos,5,carnaval cadiz,1.0
2,0pGhoB99qpEJEsBQxgaskQ,64.0,Los “Pofesionales”,7,carnaval cadiz,1.0
3,3HDrX2OtSuXLW5dLR85uN3,53.0,Los Que No Paran De Rajar,6,carnaval cadiz,1.0
4,22mLrN5fkppmuUPsHx6i2G,59.0,Vera Dulova,3,classical harp,2.0
...,...,...,...,...,...,...
305590,0vZbvoyHDSwTGY4v1oUo31,1682.0,Josh Sneed,20,clean comedy,231.0
305591,3V4ODgUObsEHFX6GQUy8xl,419.0,Matt McCarthy,9,deep comedy,2412.0
305592,3cOzi726Iav1toV2LRVEjp,4831.0,Ali Siddiq,34,black comedy,3448.0
305593,5nvjpU3Y7L6Hpe54QuvDjy,2357.0,Donnell Rawlings,15,black comedy,3448.0


In [32]:
df_artists = df_artists[df_artists['genre_id'].notna()]
df_artists = df_artists[df_artists['followers'].notna()]
df_artists = df_artists[df_artists['popularity'].notna()]
df_artists[df_artists['genre_id'].isnull()]
df_artists.columns = ['id', 'followers', 'name_art','popularity','genero', 'genre_id']
df_artists['name_art'] = df_artists['name_art'].replace({'\'': ''},regex=True)
df_artists = df_artists.reindex(columns=['id', 'followers', 'genre_id', 'name_art', 'popularity'])
df_artists

,id,followers,genre_id,name_art,popularity
0,0VLMVnVbJyJ4oyZs2L3Yl2,71.0,1.0,Las Viudas De Los Bisabuelos,6
1,0dt23bs4w8zx154C5xdVyl,63.0,1.0,Los De Capuchinos,5
2,0pGhoB99qpEJEsBQxgaskQ,64.0,1.0,Los “Pofesionales”,7
3,3HDrX2OtSuXLW5dLR85uN3,53.0,1.0,Los Que No Paran De Rajar,6
4,22mLrN5fkppmuUPsHx6i2G,59.0,2.0,Vera Dulova,3
...,...,...,...,...,...
305590,0vZbvoyHDSwTGY4v1oUo31,1682.0,231.0,Josh Sneed,20
305591,3V4ODgUObsEHFX6GQUy8xl,419.0,2412.0,Matt McCarthy,9
305592,3cOzi726Iav1toV2LRVEjp,4831.0,3448.0,Ali Siddiq,34
305593,5nvjpU3Y7L6Hpe54QuvDjy,2357.0,3448.0,Donnell Rawlings,15


In [33]:
df_artists.shape

(304411, 5)

In [34]:
df_tracks_final

,id,name_tracks,id_artists,popularity,duration_s,explicit,danceability,energy,loudness,speechiness,acousticness,tempo
0,35iwgR4jXetI318WEWsa1Q,Carve,45tIt06XoI0Iio4LBEVpls,6.0,126.903,0,0.645,0.445,13.338,0.4510,0.674,104.851
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,14jtPCOoNZwquk5wd9DxrY,0.0,98.200,0,0.695,0.263,22.136,0.9570,0.797,102.009
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,5LiOoJbxVSAMkBS2fUm3X2,0.0,181.640,0,0.434,0.177,21.180,0.0512,0.994,130.418
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,3BiJGZsyX9sJchTqcSA7Su,0.0,163.080,0,0.402,0.158,16.900,0.0390,0.989,103.220
5,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,3BiJGZsyX9sJchTqcSA7Su,0.0,178.933,0,0.227,0.261,12.343,0.0382,0.994,118.891
...,...,...,...,...,...,...,...,...,...,...,...,...
586664,1Kzjk1EyngBcP4T8x3fyqv,同行 (新加坡電視劇《愛...沒有距離》主題曲),6PWJWwEm8BSBFAIAUWlwe4,43.0,205.238,0,0.743,0.679,3.952,0.0323,0.269,126.070
586667,5rgu12WBIHQtvej2MdHSH0,云与海,1QLBXKM5GCpyQQSVMNZqrZ,50.0,258.267,0,0.560,0.518,7.471,0.0292,0.785,131.896
586668,0NuWgxEp51CutD2pJoF4OM,blind,1dy5WNgIKQU6ezkpZs4y8z,72.0,153.293,0,0.765,0.663,5.223,0.0652,0.141,150.091
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,4jGPdu95icCKVF31CcFKbS,58.0,142.003,0,0.696,0.615,6.212,0.0345,0.206,90.029


# Load

## Bigquery

In [ ]:
!pip install --upgrade google-cloud-bigquery
!pip install pandas-gbq
!pip install db-dtypes

In [38]:
import random
import string
import numpy as np
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

In [39]:
def get_random_string(length):
    # choose from all lowercase letter
    letters = string.ascii_lowercase
    return "".join(random.choice(letters) for i in range(length))

In [40]:
credentials = service_account.Credentials.from_service_account_file("/content/drive/MyDrive/segunda-entrega.json", scopes=["https://www.googleapis.com/auth/cloud-platform"])

In [41]:
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [62]:
BQ_TABLE_NAME_1 = "segunda-entrega-381117.Prueba.genders"
print(BQ_TABLE_NAME_1)
BQ_TABLE_NAME_2 = "segunda-entrega-381117.Tracks"
print(BQ_TABLE_NAME_2)
BQ_TABLE_NAME_3 = "segunda-entrega-381117.Artists"
print(BQ_TABLE_NAME_3)

segunda-entrega-381117.Prueba.genders


In [63]:
# Creating the job config
job_config = bigquery.LoadJobConfig(
    schema=[
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField("id_gender", bigquery.enums.SqlTypeNames.BIGNUMERIC),
        bigquery.SchemaField("name_gender", bigquery.enums.SqlTypeNames.STRING)
    ],
    # Drod and re-create table, if exist
    write_disposition="WRITE_TRUNCATE"
)

In [64]:
job = client.load_table_from_dataframe(df_genres, BQ_TABLE_NAME, job_config=job_config)
job.result()

ValueError: ignored

In [68]:
df_genres.to_gbq(
        destination_table="segunda-entrega-381117.Prueba.genders",
        project_id="segunda-entrega-381117",
        credentials=credentials, 
        if_exists = 'replace'
    )            

100%|██████████| 1/1 [00:00<00:00, 652.30it/s]


In [71]:
df_artists.to_gbq(
        destination_table="segunda-entrega-381117.Prueba.artists",
        project_id="segunda-entrega-381117",
        credentials=credentials, 
        if_exists = 'replace'
    )            

100%|██████████| 1/1 [00:00<00:00, 8019.70it/s]


In [72]:
df_tracks_final.to_gbq(
        destination_table="segunda-entrega-381117.Prueba.tracks",
        project_id="segunda-entrega-381117",
        credentials=credentials, 
        if_exists = 'replace'
    )            

100%|██████████| 1/1 [00:00<00:00, 4232.40it/s]
